Examples for the use of Iterative Maximum Likelihood Estimation, as well as the pauli_values extraction method, integrated both in analysis_v2 and analysis_v3. The first is an iterative algorithm that reconstructs physical matrices faster than the maximum likelihood algorithm, based on the POVM that we use for tomography measurements. The second one extracts the measured pauli values directly from the data while taking into account the assignment correction done by the calibration points.

All results are loaded from Qudev Share, 'Q:\USERS\SergiM\MLE', but can also be found in the respective setup pydata shares using the timestamps. The plots are automatically saved in the data folder; feel free to erase the existing ones to check that each cell generates what you expect (v2 analysis overwrites previous plot even if the method is changed). Everything is backed up.

Integration in analysis_v3 is done with results from XLD in the first block, where I also include an example with v2.

In the Bluefors1 block, we can find v2 examples on the sets that the method performance was tested with (2qb and 3qb states).

# XLD

Load virtual machine with parameters from XLD, to perform analysis on results from XLD experiments.

In [ ]:
from pycqedscripts.init.xld.virtual_ATC75_M136_S17HW02_PQSC import *

import warnings
warnings.filterwarnings("ignore")

# If running into problems with the initalization, it could be the AWG wave loading lines at the end of the init file.
# Commenting out: 

# for AWG in AWGs:
#     pulsar.set(f'{AWG.name}_use_placeholder_waves', True)
# pulsar.use_sequence_cache(True)

# in lines 1096-1098 (on the last revision in 04/2021) should solve the issue

Example of use in v3 is based on do_state_tomo_analysis function in pycqedscripts.scripts.characterization.state_tomo:

In [ ]:
import traceback
import qutip as qtp
import collections
odict = collections.OrderedDict
from pycqed.analysis_v3 import *

def do_state_tomo_analysis(timestamp, meas_obj_names=None, rho_target=None,
                           estimation_types=None, iterations=None, tolerance=None,
                           save_processed_data=True, save_figures=True):
    """
    meas_obj_names: list of qb names. If not provided, will be taken from task_list
    """
    pp = pp_mod.ProcessingPipeline(add_param_method='replace')
    try:
        if estimation_types is None:
            estimation_types=('least_squares', 'max_likelihood')
        task_list = hlp_mod.get_param_from_metadata_group(timestamp, 'task_list')
        if rho_target is None:
            rho_target = qtp.Qobj(task_list[0]['rho_target'])
        if meas_obj_names is None:
            meas_obj_names = task_list[0]['qubits']
        movnm = hlp_mod.get_param_from_metadata_group(timestamp, 'meas_obj_value_names_map')
        final_rots_basis = hlp_mod.get_param_from_metadata_group(timestamp, 'final_rots_basis')

        params_dict = {}
        params_dict.update({f'{qbn}.state_prob_mtx': f'Instrument settings.{qbn}.acq_state_prob_mtx'
                            for qbn in meas_obj_names})
        pp.add_node('extract_data_hdf', timestamps=timestamp, params_dict=params_dict)
        pp.add_node('do_postselection_f_level',
                    keys_in='raw',
                    num_keys_out=1,
                    meas_obj_names=meas_obj_names)
        pp.add_node('calculate_flat_multiqubit_shots',
                    keys_in='raw',
                    joint_processing=True,
                    do_preselection=True,
                    meas_obj_names=meas_obj_names)
        pp.add_node('average_data',
                    keys_in='previous calculate_flat_multiqubit_shots',
                    averaging_axis=0,
                    joint_processing=True,
                    meas_obj_names=meas_obj_names)
        pp.add_node('correct_readout',
                    keys_in='previous average_data',
                    joint_processing=True,
                    meas_obj_names=meas_obj_names)
        pp.add_node('extract_leakage_classified_shots',
                    keys_in='previous correct_readout',
                    joint_processing=True,
                    meas_obj_names=meas_obj_names)
        pp.add_node('state_tomography_analysis',
                    keys_in='previous do_postselection_f_level',
                    meas_obj_names=meas_obj_names,
                    keys_in_leakage=['previous extract_leakage_classified_shots'],
                    joint_processing=True,
                    basis_rots=final_rots_basis,
                    do_preselection=True,
                    rho_target=rho_target,
                    iterations=iterations,
                    tolerance=tolerance,
                    estimation_types=estimation_types)

        pp.resolve(movnm)
        pp()
        pp.save(save_processed_data=save_processed_data, save_figures=save_figures)
        return pp
    except Exception:
        traceback.print_exc()
        return pp

Load two experiment results for v2 and v3 test respectively

In [ ]:
timestamp = '20210131_201258' 

a_tools.datadir = r'Q:\USERS\SergiM\MLE'
MC.datadir(r'Q:\USERS\SergiM\MLE')

for qb in qubits:
    gen.load_settings(qb, timestamp=timestamp)
gen.load_settings(dev, timestamp=timestamp)

ts_start = '20210131_201258' 
ts_end = '20210131_222513' 

timestamps_v3 = a_tools.get_timestamps_in_range(ts_start, ts_end)
print(timestamps_v3)

rho_phi = qtp.Qobj([[1/2,0,0,1/2],[0,0,0,0],[0,0,0,0],[1/2,0,0,1/2]])

## Analysis_v3 example

New method is called 'iterative_mle'

In [ ]:
do_state_tomo_analysis(timestamps_v3[1], rho_target=rho_phi, estimation_types=['iterative_mle'])

We can also set manually the maximum iterations and the minimum tolerance for change between consecutive steps. In this example the limit will be set by iterations before the tolerance is reached.

In [ ]:
do_state_tomo_analysis(timestamps_v3[1], rho_target=rho_phi, estimation_types=['iterative_mle'], iterations=100, tolerance=-18)

After testing, a good default value for tolerance seemed -15 for 2 qubits and -18 for 3 qubits. Iterations are set to 1000 by default to give preference to tolerance limits; -18 tolerance corresponds to iterations on the order of 100 on average.

To extract pauli values directly, we can use 'pauli_values'. This method does not reconstruct physical matrices, like LST.

In [ ]:
do_state_tomo_analysis(timestamps_v3[1], rho_target=rho_phi, estimation_types=['pauli_values'])

## Analysis_v2 example

With v2 we need to load more information into the analysis function. This can be found in the experiment hdf file.

In [ ]:
rots_basis=('I', 'X180', 'Y90', 'mY90', 'X90', 'mX90')
use_cal_points = True
preselection = True
qbs = (qb10, qb9)
measure_qubits = [qb10,qb9]
num_qubits = len(measure_qubits)
meas_obj_names = ['qb10','qb9']
thresholds = {qb.name: qb.acq_classifier_params()['thresholds'][0] for qb in measure_qubits}
for qb in measure_qubits:
    qb.update_detector_functions()
channel_map = {qb.name: qb.int_log_det.value_names[0] + f' {qb.instr_uhf()}' for qb in measure_qubits}
n_segments = len(rots_basis)**num_qubits + (2**num_qubits if use_cal_points else 0)

Unlike v3, the method is specified with a keyworded parameter 'name_of_method'=True. The hierarchy of preference in the methods is 'pauli_values' > 'pauli_raw'* > 'imle' > 'mle'; this order is also specified in the code.

*older method for direct pauli extraction without assignment correction

In [ ]:
ts = timestamps_v3[0]

start_time = time.time()

MA = tda.StateTomographyAnalysis(t_start=ts, options_dict=dict(
    n_readouts=(2 if preselection else 1)*n_segments,
    thresholds=odict([(qb.name, thresholds[qb.name]) for qb in qbs]),
    channel_map=odict([(qb.name, channel_map[qb.name]) for qb in qbs]),
    cal_points=[odict([(channel_map[qb.name], [2*i+1 if preselection else i]) 
                       for qb in measure_qubits]) for i in np.arange(-2**num_qubits, 0)],
    data_type='singleshot',
    rho_target=rho_phi,
    basis_rots_str=rots_basis,
    covar_matrix=np.diag(np.ones(2**num_qubits)),
    imle=True,
    use_preselection=preselection,
    data_filter=(lambda data: data[1:2*len(rots_basis)**num_qubits+1:2]) if preselection \
        else (lambda data: data[:len(rots_basis)**num_qubits])
))
print(time.time()-start_time)

In [ ]:
ts = timestamps_v3[0]

start_time = time.time()

MA = tda.StateTomographyAnalysis(t_start=ts, options_dict=dict(
    n_readouts=(2 if preselection else 1)*n_segments,
    thresholds=odict([(qb.name, thresholds[qb.name]) for qb in qbs]),
    channel_map=odict([(qb.name, channel_map[qb.name]) for qb in qbs]),
    cal_points=[odict([(channel_map[qb.name], [2*i+1 if preselection else i]) 
                       for qb in measure_qubits]) for i in np.arange(-2**num_qubits, 0)],
    data_type='singleshot',
    rho_target=rho_phi,
    basis_rots_str=rots_basis,
    covar_matrix=np.diag(np.ones(2**num_qubits)),
    pauli_values=True,
    use_preselection=preselection,
    data_filter=(lambda data: data[1:2*len(rots_basis)**num_qubits+1:2]) if preselection \
        else (lambda data: data[:len(rots_basis)**num_qubits])
))
print(time.time()-start_time)

# Bluefors1

Load virtual machine with parameters from Bluefors1, to perform analysis on results from BF1 experiments.

In [ ]:
from pycqedscripts.init.bluefors1.ATC81_M138_S705_test import *

init_dict = initialize_setup(virtual_setup=True)
globals().update(**init_dict)
clear_output()

import csv
import warnings
warnings.filterwarnings("ignore")

startup = True

## Analysis on 2 qubits

Load 6 bell state tomographies on different qubit pairs of a BF1 chip.

In [ ]:
a_tools.datadir = r'Q:\USERS\SergiM\MLE'
MC.datadir(r'Q:\USERS\SergiM\MLE')

ts_start = '20210210_140836' 
ts_end = '20210210_142102' 

timestamps = a_tools.get_timestamps_in_range(ts_start, ts_end)
    
for qb in qubits:
    gen.load_settings(qb, timestamp=timestamps[0])
if startup:
    dev.add_2qb_gate('CZ_nztc', 'NZTransitionControlledPulse')
    startup = False
gen.load_settings(dev, timestamp=timestamps[0])

print(timestamps)

Load parameters needed for v2 analysis. These can be found and extracted from the hdf measurement file if they are not known. I also define the target state.

In [ ]:
rots_basis=('I', 'X180', 'Y90', 'mY90', 'X90', 'mX90')
use_cal_points = True
preselection = False
qbs_list = [(qb3, qb1),(qb3, qb6),(qb4, qb1),(qb4, qb7),(qb5, qb2),(qb5, qb7)]
measure_qubits_list = [[qb3, qb1],[qb3, qb6],[qb4, qb1],[qb4, qb7],[qb5, qb2],[qb5, qb7]]
num_qubits = 2

thresholds = {qb.name: qb.acq_classifier_params()['thresholds'][0] for qb in qubits}
for qb in qubits:
    qb.update_detector_functions()
channel_map = {qb.name: qb.int_log_det.value_names[0] + f' {qb.instr_uhf()}' for qb in qubits}
n_segments = len(rots_basis)**2 + (2**2 if use_cal_points else 0)

rho_phi = qtp.Qobj([[1/2,0,0,1/2],[0,0,0,0],[0,0,0,0],[1/2,0,0,1/2]])

Examples with IMLE method on n of the 6 loaded examples:

In [ ]:
n = 1

In [ ]:
for i, ts in enumerate(timestamps[:n]):
    print(i)
    measure_qubits = measure_qubits_list[i]
    qbs = qbs_list[i]
    rho_target = rho_phi
    
    start_time = time.time()
    num_qubits = len(measure_qubits)

    MA = tda.StateTomographyAnalysis(t_start=ts, options_dict=dict(
        n_readouts=(2 if preselection else 1)*n_segments,
        thresholds=odict([(qb.name, thresholds[qb.name]) for qb in qbs]),
        channel_map=odict([(qb.name, channel_map[qb.name]) for qb in qbs]),
        cal_points=[odict([(channel_map[qb.name], [2*i+1 if preselection else i]) 
                           for qb in measure_qubits]) for i in np.arange(-2**num_qubits, 0)],
        data_type='singleshot',
        rho_target=rho_target,
        basis_rots_str=rots_basis,
        imle = True,
        covar_matrix=np.diag(np.ones(2**num_qubits)),
        use_preselection=preselection,
        data_filter=(lambda data: data[1:2*len(rots_basis)**num_qubits+1:2]) if preselection \
            else (lambda data: data[:len(rots_basis)**num_qubits])
    ))

We can also set arbitrary tolerance and iteration limits like this (Effect and default values are explained in the XLD block).

In [ ]:
for i, ts in enumerate(timestamps[:n]):
    print(i)
    measure_qubits = measure_qubits_list[i]
    qbs = qbs_list[i]
    rho_target = rho_phi
    
    start_time = time.time()
    num_qubits = len(measure_qubits)

    MA = tda.StateTomographyAnalysis(t_start=ts, options_dict=dict(
        n_readouts=(2 if preselection else 1)*n_segments,
        thresholds=odict([(qb.name, thresholds[qb.name]) for qb in qbs]),
        channel_map=odict([(qb.name, channel_map[qb.name]) for qb in qbs]),
        cal_points=[odict([(channel_map[qb.name], [2*i+1 if preselection else i]) 
                           for qb in measure_qubits]) for i in np.arange(-2**num_qubits, 0)],
        data_type='singleshot',
        rho_target=rho_target,
        basis_rots_str=rots_basis,
        imle = True,
        iterations = 100,
        tolerance = -18,
        covar_matrix=np.diag(np.ones(2**num_qubits)),
        use_preselection=preselection,
        data_filter=(lambda data: data[1:2*len(rots_basis)**num_qubits+1:2]) if preselection \
            else (lambda data: data[:len(rots_basis)**num_qubits])
    ))

Examples with pauli_values :

In [ ]:
for i, ts in enumerate(timestamps[:n]):
    print(i)
    measure_qubits = measure_qubits_list[i]
    qbs = qbs_list[i]
    rho_target = rho_phi
    
    start_time = time.time()
    num_qubits = len(measure_qubits)

    MA = tda.StateTomographyAnalysis(t_start=ts, options_dict=dict(
        n_readouts=(2 if preselection else 1)*n_segments,
        thresholds=odict([(qb.name, thresholds[qb.name]) for qb in qbs]),
        channel_map=odict([(qb.name, channel_map[qb.name]) for qb in qbs]),
        cal_points=[odict([(channel_map[qb.name], [2*i+1 if preselection else i]) 
                           for qb in measure_qubits]) for i in np.arange(-2**num_qubits, 0)],
        data_type='singleshot',
        rho_target=rho_target,
        basis_rots_str=rots_basis,
        pauli_values = True,
        covar_matrix=np.diag(np.ones(2**num_qubits)),
        use_preselection=preselection,
        data_filter=(lambda data: data[1:2*len(rots_basis)**num_qubits+1:2]) if preselection \
            else (lambda data: data[:len(rots_basis)**num_qubits])
    ))

## Analysis on 3 qubits

Load 9 data sets for tomographies of 3-qubit states.

In [ ]:
timestamp = '20201208_140754' 

a_tools.datadir = r'Q:\USERS\SergiM\MLE'
MC.datadir(r'Q:\USERS\SergiM\MLE')

for qb in qubits:
    gen.load_settings(qb, timestamp=timestamp)

cz_type = 'CZ_nztc'
if startup:
    dev.add_2qb_gate('CZ_nztc', 'NZTransitionControlledPulse')
    startup = False
gen.load_settings(dev, timestamp=timestamp)

ts_start = '20201208_140754' 
ts_end = '20201208_144607' 

timestamps = a_tools.get_timestamps_in_range(ts_start, ts_end)

Load parameters needed for v2 analysis. These can be found and extracted from the hdf measurement file if they are not known

In [ ]:
rots_basis=('I', 'X180', 'Y90', 'mY90', 'X90', 'mX90')
use_cal_points = True
preselection = True
qbs = (qb1, qb3, qb6)
measure_qubits = [qb1, qb3, qb6]
num_qubits = len(measure_qubits)
meas_obj_names = ['qb1','qb3','qb6']
thresholds = {qb.name: qb.acq_classifier_params()['thresholds'][0] for qb in measure_qubits}
for qb in measure_qubits:
    qb.update_detector_functions()
channel_map = {qb.name: qb.int_log_det.value_names[0] + f' {qb.instr_uhf()}' for qb in measure_qubits}
n_segments = len(rots_basis)**3 + (2**3 if use_cal_points else 0)

This reads the prepared states from the experiment files, which correspond to some ground states of a quantum phase recognition experiment and are not trivial to write like the 2 qb Bell state.

In [ ]:
def state_preparation(num_qubits,param_set):

    g_st = qtp.basis(2,0)
    if num_qubits == 3:
        initial = qtp.tensor(g_st,g_st,g_st)
        rot1 = qtp.tensor(qtp.qip.operations.ry(param_set[0]),qtp.qip.operations.ry(param_set[1]),qtp.qeye(2))
        cz1 = qtp.tensor(qtp.qip.operations.csign(),qtp.qeye(2))
        rot2 = qtp.tensor(qtp.qip.operations.ry(param_set[2]), qtp.qip.operations.ry(param_set[3]), qtp.qip.operations.ry(param_set[4]))
        cz2 = qtp.tensor(qtp.qeye(2),qtp.qip.operations.csign())
        rot3 = qtp.tensor(qtp.qeye(2), qtp.qip.operations.ry(param_set[5]), qtp.qip.operations.ry(param_set[6]))
    elif num_qubits == 7:
        initial = qtp.tensor(g_st,g_st,g_st,g_st,g_st,g_st,g_st)
        rot1 = qtp.tensor(qtp.ry(param_set[0]),qtp.ry(param_set[1]),qtp.ry(param_set[2]),qtp.ry(param_set[3]),qtp.ry(param_set[4]),qtp.ry(param_set[5]),qeye(2))
        cz1 = qtp.tensor(qtp.csign(),qtp.csign(),qtp.csign(),qtp.qeye(2))
        rot2 = qtp.tensor(qtp.ry(param_set[6]),qtp.ry(param_set[7]),qtp.ry(param_set[8]),qtp.ry(param_set[9]),qtp.ry(param_set[10]),qtp.ry(param_set[11]),qtp.ry(param_set[12]))
        cz2 = qtp.tensor(qtp.qeye(2),qtp.csign(),qtp.csign(),qtp.csign())
        rot3 = qtp.tensor(qtp.qeye(2),qtp.ry(param_set[13]),qtp.ry(param_set[14]),qtp.ry(param_set[15]),qtp.ry(param_set[16]),qtp.ry(param_set[17]),qtp.ry(param_set[18]))

    psi_target = rot3 * cz2 * rot2 * cz1 * rot1 * initial

    rho_target = (psi_target * psi_target.dag()).full()
    
    return rho_target

# Read the generated states from the csv files
param_set = []
rho_target_list = []
try:
    for ts in timestamps:
        read_data = []
        with open(a_tools.data_from_time(ts) + "\\OptParams.csv") as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                read_data.append(row)
            param = [float(var) for var in read_data[0]]
        param_set.append(param)
        exp_target = qtp.Qobj(state_preparation(num_qubits, param), dims=[[2,2,2],[2,2,2]])
        rho_target_list.append(exp_target)
except FileNotFoundError:
    print('file not found. Using default rho comparison')
    J = 1.0
    h2_list = np.linspace(-1.6, 1.6, 9)
    h1_list = [0.1]

    fid_list = []
    rho_target_list = []
    exp_list = []
    i=0
    for h1 in h1_list:
        for h2 in h2_list:
            # Target ground state
            psi_target = H_qpr_s(J, h1, h2).groundstate()[1]
            rho_target = psi_target * psi_target.dag()
            rho_target_list.append(rho_target)

Imle and pauli tests on n of the 9 states of the experiment.

In [ ]:
n = 1

In [ ]:
for j, ts in enumerate(timestamps[:n]):
    print(j)
    start_time = time.time()

    rho_target = rho_target_list[j]

    MA = tda.StateTomographyAnalysis(t_start=ts, options_dict=dict(
        n_readouts=(2 if preselection else 1)*n_segments,
        thresholds=odict([(qb.name, thresholds[qb.name]) for qb in qbs]),
        channel_map=odict([(qb.name, channel_map[qb.name]) for qb in qbs]),
        cal_points=[odict([(channel_map[qb.name], [2*i+1 if preselection else i]) 
                           for qb in measure_qubits]) for i in np.arange(-2**num_qubits, 0)],
        data_type='singleshot',
        rho_target=rho_target,
        basis_rots_str=rots_basis,
        covar_matrix=np.diag(np.ones(2**num_qubits)),
        imle=True,
        use_preselection=preselection,
        data_filter=(lambda data: data[1:2*len(rots_basis)**num_qubits+1:2]) if preselection \
            else (lambda data: data[:len(rots_basis)**num_qubits])
    ))
    print(time.time()-start_time)

In [ ]:
for j, ts in enumerate(timestamps[:1]):
    print(j)
    start_time = time.time()
    
    num_qubits = 3
    rho_target = rho_target_list[j]

    MA = tda.StateTomographyAnalysis(t_start=ts, options_dict=dict(
        n_readouts=(2 if preselection else 1)*n_segments,
        thresholds=odict([(qb.name, thresholds[qb.name]) for qb in qbs]),
        channel_map=odict([(qb.name, channel_map[qb.name]) for qb in qbs]),
        cal_points=[odict([(channel_map[qb.name], [2*i+1 if preselection else i]) 
                           for qb in measure_qubits]) for i in np.arange(-2**num_qubits, 0)],
        data_type='singleshot',
        rho_target=rho_target,
        basis_rots_str=rots_basis,
        covar_matrix=np.diag(np.ones(2**num_qubits)),
        pauli_values=True,
        use_preselection=preselection,
        data_filter=(lambda data: data[1:2*len(rots_basis)**num_qubits+1:2]) if preselection \
            else (lambda data: data[:len(rots_basis)**num_qubits])
    ))
    print(time.time()-start_time)

We can also set arbitrary iterations and tolerance for the method following the example with 2 qubits.